In [1]:
import pandas as pd
import numpy as np

# 鄰里資料彙整

-以"村里代碼對照表"為主資料及與其他做合併

1. 村里代碼對照表(n=2140)
        - 106年行政院主計總處與內政部戶政司村里代碼對照表
2. 鄰里辦公室資料 (n=2149)
        - 2020年里長通訊錄
3. 鄰里人口結構  (n=2149)
        - 全國人口密度_村里_108年
4. 2018年的所得資料(n=2149)
        - '全台所得地圖'
5. 電子發票消費熱度指標 (n=7244,2020年)
        - 2017-2020年電子發票消費熱度指標(2020.07)

-行政院主計總處代碼已廢棄，改以內政部戶政司為主<br> 
-106年之後桃園地區鄰里多了9個   https://news.ltn.com.tw/news/local/paper/1112757https://news.ltn.com.tw/news/local/paper/1112757<br> 
-人口密度的鄰里欄位中文字碼無法正確解析，但村里代碼完整<br> 
-行政院主計總處的檔案少了桃園地區的9個鄰里(2020年新增)，但中文字碼解析<br> 
-'臺'統一取代為'台'

### 1. 合併108年人口資料與'全台所得地圖'2018年的所得資料 

- 人口結構資料來源 - 社會經濟資料服務平台社會經濟資料服務平台 <br> 
網站參考: https://segis.moi.gov.tw/STAT/Web/Platform/QueryInterface/STAT_QueryTopProduct.aspx

- 所得資料採用'全台所得地圖' <br>
網站參考:https://kiang.github.io/salary/map/#2018/playButton2/65000040045

- 新北與台北共有3個里因戶數過少被裁併新北與台北共有3個里因戶數過少被裁併


#### 1.1 將108年所得資料加入村里代碼
- 加上村里代碼 (取自108年人口結構資料 )

In [2]:
# 取出108年人口結構資料
df_108 = pd.read_csv('../data/raw_data/全國人口密度_村里_108年.csv',
                 header=1 , encoding='utf-8') # 108年人口結構資料   

df_108 = df_108[['縣市名稱','鄉鎮市區名稱','村里代碼', '村里名稱']]
df_108 = df_108.rename(columns={'縣市名稱':'Country','鄉鎮市區名稱':'District',
                                '村里代碼':'Village_ID','村里名稱':'Village'})

df_108['Village'] = df_108['Village'].str.replace('臺','台')
df_108['Country'] = df_108['Country'].str.replace('臺','台')

df_108 = df_108[(df_108['Country']=='台北市') | 
                (df_108['Country']=='新北市') | 
                (df_108['Country']=='桃園市') |
                (df_108['Country']=='基隆市')]      # n=2149
print('人口結構資料數量:\t',df_108.index.size)


# 取出收入資料
df_income = pd.read_csv('../data/raw_data/2018各區里綜合所得.csv', encoding='utf-8')

df_income = df_income[(df_income['縣市']=='台北市') |
                      (df_income['縣市']=='新北市') |
                      (df_income['縣市']=='桃園市') |
                      (df_income['縣市']=='基隆市')] 

df_income = df_income[df_income['納稅單位']>1] #有3個里因為戶數太少被裁併，因此直接排除
print('收入資料數量:\t',df_income.index.size)  # 2149

# df_income.columns ='納稅單位', '綜合所得總額', '平均數', '中位數', '第一分位數', '第三分位數', '標準差', '變異係數'

df_income = df_income[['縣市','鄉鎮市區', '村里','納稅單位', '綜合所得總額', '平均數', '中位數']]

# 合併資料集  -採用行政區名比對
df_income = pd.merge(df_income, df_108, 
             how ='outer',
             left_on  = ['縣市','鄉鎮市區', '村里'] ,
             right_on = ['Country', 'District', 'Village'])   ## 注意要同時採用3個行政區做合併，因為不同縣市有相同的區里名

人口結構資料數量:	 2149
收入資料數量:	 2149


In [3]:
# 填補缺值
"""注意指定值的時候避免多重索引方式(chain indexing)"""

## 取得108年人口密度的 村里代碼 
pop_108 = df_income[df_income['縣市'].isnull()]                        # 取得沒有成功合併的資料中，'108年人口密度'的資料
village_ID = pop_108.sort_values(by='Village')['Village_ID'].copy()   # 資料排序後 複製人口密度資料的村里代碼值  

## 取得收入資料缺值的index位置
income_null = df_income[df_income['Village'].isnull()]                 # 取得沒有成功合併的資料中， '收入資料'缺值的部分
index = income_null.sort_values(by='村里').index.copy()                 # 資料排序取得index值

df_income.loc[index,'Village_ID'] = village_ID.values                  # 叫出缺值進行填補  
print('合併後的資料數量:\t\t' , df_income.index.size) # return 2161
# df.loc[index]   # check null value

# 刪除多餘欄位與缺值
df_income.drop(columns=['Country','District','Village'], inplace=True)
df_income.dropna(inplace=True)
print('整理後的收入資料數量:\t',df_income.index.size) # return 2149

df_income = df_income.rename(columns={'縣市':'Country','鄉鎮市區':'District','村里':'Village',
                                     '平均數':'所得平均', '中位數':'所得中位'})

df_income[['綜合所得總額','所得平均', '所得中位']] = df_income[['綜合所得總額','所得平均', '所得中位']].apply(lambda  x : x/10)

df_income = df_income[['Country', 'District', 'Village','Village_ID', '納稅單位','綜合所得總額', '所得平均']]

df_income.to_csv('../data/df_income', encoding='utf-8')
df_income

合併後的資料數量:		 2168
整理後的收入資料數量:	 2149


,Country,District,Village,Village_ID,納稅單位,綜合所得總額,所得平均
0,台北市,松山區,中崙里,63000010-027,1410.0,216020.1,153.2
1,台北市,松山區,自強里,63000010-021,3224.0,449470.0,139.4
2,台北市,松山區,鵬程里,63000010-020,1850.0,227937.3,123.2
3,台北市,松山區,東榮里,63000010-003,2923.0,485124.1,166.0
4,台北市,松山區,介壽里,63000010-009,1554.0,240226.0,154.6
...,...,...,...,...,...,...,...
2144,桃園市,復興區,高義里,68000130-010,117.0,7449.6,63.7
2145,桃園市,復興區,三光里,68000130-006,90.0,5764.7,64.1
2146,桃園市,復興區,奎輝里,68000130-007,148.0,9804.8,66.2
2147,桃園市,復興區,華陵里,68000130-008,176.0,12544.1,71.3


#### 1.2 整合108年人口結構與2018年收入資料

In [11]:
df_108 = pd.read_csv('../data/raw_data/全國人口密度_村里_108年.csv',
                 header=1 , encoding='utf-8') # 108年人口結構資料   

# 抓出108年人口結構欄位
columns_select = ['縣市名稱','鄉鎮市區名稱','村里名稱','村里代碼', '戶數', '人口數','男性人口數', '女性人口數',
                  '0-14歲人口數', '15-64歲人口數', '65歲以上人口數','15-19歲人口數', '20-24歲人口數',
                  '25-29歲人口數', '30-34歲人口數', '35-39歲人口數', '40-44歲人口數', '45-49歲人口數',
                  '50-54歲人口數', '55-59歲人口數', '60-64歲人口數',  
                  '15歲以上博士人口數', '15歲以上碩士人口數', '15歲以上大學院校人口數','15歲以上專科人口數',
                  '15歲以上高中職人口數', '15歲以上國中初職人口數', '15歲以上小學人口數','15歲以上不識字人口數', 
                  '15歲以上未婚人口數', '15歲以上有偶人口數','出生數','死亡數','遷入人口數', '遷出人口數',
                  ]
df_108 = df_108[columns_select]

df_108 = df_108.rename(columns={'縣市名稱':'Country','鄉鎮市區名稱':'District',
                                '村里代碼':'Village_ID','村里名稱':'Village'})

# rename 欄位
df_108.columns = df_108.columns.str.replace('15歲以上','')
df_108.columns = df_108.columns.str.replace(r'人口數','人口')
df_108['Village'] = df_108['Village'].str.replace('臺','台')
df_108['Country'] = df_108['Country'].str.replace('臺','台')

df_108 = df_108[(df_108['Country']=='台北市') | 
                (df_108['Country']=='新北市') | 
                (df_108['Country']=='桃園市') |
                (df_108['Country']=='基隆市')]      # n=2149


# 將出生數量為nan的轉換為0
index = df_108[df_108['出生數'].isnull()].index.values
df_108.columns.get_loc('出生數') # return 31
df_108.loc[index,'出生數'] = 0

print(df_108.index.size)

# 合併欄位
df_pop = pd.merge(df_income, df_108.loc[:,'Village_ID':] , 
                  on='Village_ID' ,how = 'inner')
# df_pop.isnull().any()

#
df_pop.to_csv('../data/population.csv', index=False, encoding='utf-8')
df_pop.head(3)

2149


,Country,District,Village,Village_ID,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
0,台北市,松山區,中崙里,63000010-027,1410.0,216020.1,153.2,1714,4448,2079,...,716,161,151,4,2034,2010,38.0,30.0,348,349
1,台北市,松山區,自強里,63000010-021,3224.0,449470.0,139.4,3887,10100,4748,...,1683,360,367,17,4081,4729,85.0,97.0,477,555
2,台北市,松山區,鵬程里,63000010-020,1850.0,227937.3,123.2,2184,5477,2621,...,1125,235,269,18,2293,2452,29.0,45.0,232,302


### 2.合併里辦公室與人口結構資料

In [ ]:
df_pop = pd.read_csv("../data/population.csv", encoding='utf-8',) 
df_office = pd.read_csv("../data/village_office2.csv", encoding='utf-8') 

# 各資料集鄰里數量
# print('里辦公室:', df_pop.index.size)
# print('人口結構:', df_office.index.size)
# print('里辦公室:', df_pop.groupby(by=['Country']).size())
# print('人口結構:', df_office.groupby(by=['Country']).size())

df_op  = pd.merge(df_office, df_pop.loc[ : , 'Village_ID':],
                  on=['Village_ID'], how='left',
                  sort = 'Village_ID' )
# df_op.set_index('Village_ID', inplace=True)

df_op.to_csv('../data/pop_merge.csv',index=False ,encoding='utf-8')
df_op

In [13]:
df_pop.loc[ : , 'Village_ID':]

,Village_ID,納稅單位,綜合所得總額,所得平均,戶數,人口,男性人口,女性人口,0-14歲人口,15-64歲人口,...,高中職人口,國中初職人口,小學人口,不識字人口,未婚人口,有偶人口,出生數,死亡數,遷入人口,遷出人口
0,63000010-027,1410.0,216020.1,153.2,1714,4448,2079,2369,962,2887,...,716,161,151,4,2034,2010,38.0,30.0,348,349
1,63000010-021,3224.0,449470.0,139.4,3887,10100,4748,5352,1288,6725,...,1683,360,367,17,4081,4729,85.0,97.0,477,555
2,63000010-020,1850.0,227937.3,123.2,2184,5477,2621,2856,549,3960,...,1125,235,269,18,2293,2452,29.0,45.0,232,302
3,63000010-003,2923.0,485124.1,166.0,3071,8218,3827,4391,1322,5140,...,1194,239,188,5,3530,3959,50.0,36.0,435,561
4,63000010-009,1554.0,240226.0,154.6,1770,4733,2226,2507,902,2888,...,687,171,123,5,2027,2243,31.0,30.0,262,263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,68000130-010,117.0,7449.6,63.7,264,941,515,426,159,714,...,305,186,132,1,420,363,12.0,12.0,61,61
2145,68000130-006,90.0,5764.7,64.1,242,895,473,422,154,661,...,274,201,116,3,393,362,11.0,11.0,68,39
2146,68000130-007,148.0,9804.8,66.2,248,958,497,461,183,687,...,294,201,107,0,402,382,23.0,9.0,35,61
2147,68000130-008,176.0,12544.1,71.3,528,1512,855,657,233,1065,...,501,303,182,3,619,659,18.0,12.0,96,85
